In [1]:
import h5py
import numpy as np
import matplotlib
matplotlib.use('Agg')
matplotlib.rcParams['mathtext.default'] = 'regular'
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from PIL import Image
import numpy.ma as ma
from tempfile import TemporaryFile

/Users/litadlc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mp = 1.672622e-24 # mass of hydrogren atom, in grams
kb = 1.380658e-16 # boltzmann constant in ergs/K

d_c = mp # code density unit, 1 mp / cm^3
l_c = 3.08567758e18 # code length unit, 1 pc in cm
l_km = 3.08567758e13 # code length unit, 1pc in km
t_c = 3.15569e10    # code time unit, 1 kyr in seconds
v_c = l_c / t_c     # code velocity unit
v_km = l_km / t_c     # code velocity unit
p_c = d_c * v_c * v_c # code pressure unit
m_c = d_c * l_c * l_c * l_c / 1.9891e33 # code mass unit to solar masses

In [3]:
istart = 398 # starting file number
iend   = 398 # ending file number
dx = 5./64. # size scale, pc per cell
t_cc = 39.8 # cloud crushing time in kyr, n = 0.5 cloud
#t_cc = 56.4 # cloud crushing time in kyr, n = 1 cloud
dnamein='/Users/litadlc/Desktop/Data/' # directory where the file is located
dnameout='/Users/litadlc/Desktop/10tcc/' # directory where the plot will be saved

In [4]:
for i in range(istart,iend+1,30):
    print(i)
f = h5py.File(dnamein+str(i)+'.h5', 'r') # open the hdf5 file for reading
head = f.attrs # read the header attributes into structure, head
gamma = head['gamma'][0] # ratio of specific heats
t  = head['t'][0] # time of this snapshot, in kyr
nx = head['dims'][0] # number of cells in the x direction
ny = head['dims'][1] # number of cells in the y direction
nz = head['dims'][2] # number of cells in the z direction
d  = f['density'][:]
mx = f['momentum_x'][:]
#E = f['Energy'][:]
GE = f['GasEnergy'][:] # internal energy in code units, d*e
f.close()

398


In [5]:
mu = 1.0 # mean molecular weight (mu) of 1
den = d*d_c # to convert from code units to cgs, multiply by the code unit for that variable)
f_h=0.74 #solar metallicity
nFe_nH=2.82e-5
nSi_nH=3.47e-5
nO_nH=4.90e-4
nC_nH=2.45e-4
nAl_nH=2.95e-6

n_Fe = f_h*(den/(mu*mp))*(nFe_nH)
n_Si = f_h*(den/(mu*mp))*(nSi_nH)
n_O = f_h*(den/(mu*mp))*(nO_nH)
n_C = f_h*(den/(mu*mp))*(nC_nH)
n_Al = f_h*(den/(mu*mp))*(nAl_nH)

In [6]:
T = (GE*(gamma-1.0)*p_c)/(d*kb)
log_T = np.log10(T)

In [7]:
FeII = ma.masked_outside(log_T,4.05,4.15)
SiII = ma.masked_outside(log_T,3.92,4.08)
OVI = ma.masked_outside(log_T,5.4,5.5)
CII = ma.masked_outside(log_T,4.35,4.48)

In [8]:
FeII_mask = ma.getmask(FeII)
SiII_mask = ma.getmask(SiII)
OVI_mask = ma.getmask(OVI)
CII_mask = ma.getmask(CII)

In [9]:
n_Fe[FeII_mask] = 10e-20
n_Si[SiII_mask] = 10e-20
n_O[OVI_mask] = 10e-20
n_C[CII_mask] = 10e-20

In [10]:
dr=4.94e20 #in cm

In [11]:
N_FeII = np.sum(n_Fe, axis=0)*dr
N_FeII_log=np.log10(N_FeII)
N_SiII = np.sum(n_Si, axis=0)*dr
N_SiII_log=np.log10(N_SiII)
N_OVI = np.sum(n_O, axis=0)*dr
N_OVI_log=np.log10(N_OVI)
N_CII = np.sum(n_C, axis=0)*dr
N_CII_log=np.log10(N_CII)

In [12]:
print("n range FeII   = %e %e" % (np.min(N_FeII),np.max(N_FeII)))
print("n range SiII   = %e %e" % (np.min(N_SiII),np.max(N_SiII)))
print("n range OVI   = %e %e" % (np.min(N_OVI),np.max(N_OVI)))
print("n range CII   = %e %e" % (np.min(N_CII),np.max(N_CII)))

print("N range FeII   = %5.2f %5.2f" % (np.min(N_FeII_log), np.max(N_FeII_log)))
print("N range SiII  = %5.2f %5.2f" % (np.min(N_SiII_log), np.max(N_SiII_log)))
print("N range OVI   = %5.2f %5.2f" % (np.min(N_OVI_log), np.max(N_OVI_log)))
print("N range CII   = %5.2f %5.2f" % (np.min(N_CII_log), np.max(N_CII_log)))

n range FeII   = 4.937182e+01 3.743372e+14
n range SiII   = 1.011732e+05 1.391402e+18
n range OVI   = 1.011732e+05 4.869350e+17
n range CII   = 1.011732e+05 8.859425e+17
N range FeII   =  1.69 14.57
N range SiII  =  5.01 18.14
N range OVI   =  5.01 17.69
N range CII   =  5.01 17.95


In [13]:
pn_min_FeII=11
pn_max_FeII=14.57

pn_min_SiII=11
pn_max_SiII=18.14

pn_min_OVI=11
pn_max_OVI=17.69

pn_min_CII=11
pn_max_CII=17.95

In [14]:
np.save('N_SiII_log',N_SiII_log)
np.save('N_OVI_log',N_OVI_log)
np.save('N_CII_log',N_CII_log)

In [15]:
np.save('N_SiII',n_Si)
np.save('N_OVI',n_O)
np.save('N_CII',n_C)

In [16]:
N_FeII_x=n_Fe[:,260,260]
N_SiII_x=n_Si[:,260,260]
N_OVI_x=n_O[:,260,260]

In [17]:
vx = ((mx)/(d))*v_km
vx_260260=vx[250,:,:]

In [18]:
np.save('vx',vx)

In [19]:
v_hist_test = np.reshape(vx_260260, ny*nz)

In [20]:
fig =plt.figure(figsize=(11,12), frameon=False)
ax = fig.add_axes([0,0,1,1])
ax.set_xticks(2048*np.arange(0.1, 1, 0.1))
ax.set_yticks(512*np.arange(0.25, 1, 0.25))
ax.tick_params(axis='both', which='both', color='black', direction='in', labelleft=0, labelbottom=0, top=1, right=1,width=2)
vx_slice=vx[:,:,256]
plt.imshow(vx_slice.T,cmap='viridis',origin='lower')
matplotlib.rcParams.update({'font.size': 15})

matplotlib.rc('xtick', labelsize=15) 
cd=plt.colorbar(label='velocity $km/s$',orientation='horizontal')

ax.text(64, 420, r'$\tilde{n} = 0.5$ cloud ', color='black')
ax.text(1800, 420, str(int(t/t_cc))+'$t_{cc}$', color='black')
ax.text(1400, 75, r'2048x512x512 cells', color='black')
plt.savefig(dnameout+'Slice_vx'+str(i)+'.png', dpi=300)
plt.close(fig)
plt.show()

In [21]:
# numpy histogram functions like 1D arrays
#n_hist = np.reshape(log_n, nx_10*ny_10*nz_10)
mass = d*(dx**3)*m_c # mass in solar masses
v_hist = np.reshape(vx, nx*ny*nz)
m_hist = np.reshape(mass, nx*ny*nz)

In [22]:
xbins = np.linspace(0,1500,50)
fig = plt.figure(figsize=(9,8))
ax = fig.add_axes([0.19,0.17,0.58,0.75])
n, bins, patches = ax.hist(v_hist,xbins,histtype = 'step',alpha=0.9,linewidth=3.0, color = 'purple',weights=m_hist)
matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20)
ax.set_xlabel('velocity $km/s$',fontsize=20)
ax.set_ylabel('Total mass $M_{\odot}$',fontsize=20)
#ax.set_xlim(0,1500)
# save the figure
plt.savefig(dnameout+'1D_vel_m_'+str(i)+'.png', dpi=300)
plt.close(fig)

In [23]:
xbins = np.linspace(0,1500,50)
fig = plt.figure(figsize=(9,8))
ax = fig.add_axes([0.19,0.17,0.58,0.75])
n, bins, patches = ax.hist(v_hist,xbins,histtype = 'step',alpha=0.9,linewidth=3.0, color = 'black')
matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20)
ax.set_xlabel('velocity $km/s$',fontsize=20)
ax.set_ylabel('Volume $cm^{3}$',fontsize=20)

# save the figure
plt.savefig(dnameout+'1D_vel_vol_'+str(i)+'.png', dpi=300)
plt.close(fig)

In [24]:
# to create a projection, integrate the density array along one axis
n = den/(mu*mp)
pn_x = np.sum(n, axis=0)*dx*l_c
pn_y = np.sum(n, axis=1)*dx*l_c

In [25]:
# set the surface density scale
log_pn_x = np.log10(pn_x)
log_pn_y = np.log10(pn_y)
print("n range    = %e %e" % (np.min(n),np.max(n)))
print("N range    = %5.2f %5.2f" % (np.min(log_pn_x), np.max(log_pn_x)))
pn_min=18.0
pn_max=21.0

n range    = 5.868804e-04 5.532877e+02
N range    = 18.30 20.85


In [26]:
# plot the surface density, x-axis projection
fig = plt.figure(figsize=(10,10))
a0 = fig.add_axes([0,0,0.67,1.0])
a0.set_xticks(512*np.arange(0.25, 1, 0.25))
a0.set_yticks(512*np.arange(0.25, 1, 0.25))
a0.tick_params(axis='both', which='both', direction='in', color='white', labelleft=0, labelbottom=0, top=1, right=1,width=2)
image = a0.imshow(log_pn_x.T, origin='lower', cmap='viridis', vmin=pn_min, vmax=pn_max)

# add a circle showing the original extent of the cloud
circle=plt.Circle((256,256),64,fill=False,edgecolor='white',linestyle='dashed',linewidth=2.0)
fig.gca().add_artist(circle)

# add a colorbar
cbaxes = fig.add_axes([0.7, 0.17, 0.03, .66])
cb = plt.colorbar(image, cax = cbaxes, ticks=[np.arange(18.2, 21.0, 0.5)])
cbaxes.tick_params(axis='y', direction='in',width=2)
cb.solids.set_edgecolor('face')
cbaxes.set_ylabel('$log_{10}(N_{H})$ [$cm^{-2}$]')
plt.show()

# save the figure
plt.savefig(dnameout+'sd_x_'+str(i)+'.png', dpi=300)
plt.close(fig)

In [27]:
np.save('n_FeII',n_Fe)
np.save('n_SiII',n_Si)
np.save('n_OVI',n_O)

In [28]:
# plot the surface density, y-axis projection
fig = plt.figure(figsize=(10,7), frameon=False)
a0 = fig.add_axes([0,0,1,1])
a0.set_xticks(2048*np.arange(0.1, 1, 0.1))
a0.set_yticks(512*np.arange(0.25, 1, 0.25))
a0.tick_params(axis='both', which='both', color='white', direction='in', labelleft=0, labelbottom=0, top=1, right=1,width=2)
image = a0.imshow(log_pn_y.T, origin='lower', cmap='viridis', vmin=pn_min, vmax=pn_max)
a0.hlines(73, 64, 64+128, color='white')
a0.text(64+140, 64, '10 pc', color='white')
a0.text(64, 420, r'$\tilde{n} = 0.5$ cloud ', color='white')
a0.text(1800, 420, str(int(t/t_cc))+'$t_{cc}$', color='white')
plt.show()

plt.savefig(dnameout+'sd_y_'+str(i)+'.png', dpi=300)
plt.close(fig)

In [29]:
log_temp_1DH_10=np.reshape(log_T,nx*ny*nz)

In [30]:
midT_SiII=np.where(np.logical_and(log_temp_1DH_10>=3.92, log_temp_1DH_10<=4.08))
midT_OVI=np.where(np.logical_and(log_temp_1DH_10>=5.4, log_temp_1DH_10<=5.5))
#midT_FeII=np.where(np.logical_and(log_temp_1DH_10>=4.05, log_temp_1DH_10<=4.15))
#midT_CII=np.where(np.logical_and(log_temp_1DH_10>=4.3, log_temp_1DH_10<=4.45))

In [31]:
xbins = np.linspace(0,1200,200)
fig = plt.figure(figsize=(15,15))
ax = fig.add_axes([0.19,0.17,0.58,0.75])
#v_hist_midT_SiII=v_hist[midT_SiII]
v_hist_midT_OVI=v_hist[midT_OVI]
#v_hist_midT_FeII=v_hist[midT_FeII]
#v_hist_midT_CII=v_hist[midT_CII]
#m_hist_midT_SiII=m_hist[midT_SiII]
m_hist_midT_OVI=m_hist[midT_OVI]
#m_hist_midT_FeII=m_hist[midT_FeII]
#m_hist_midT_CII=m_hist[midT_CII]
#plt.hist(v_hist_midT_SiII,xbins,histtype = 'step', alpha=0.9,color='rematpd',label ='Si II',weights=m_hist_midT_SiII)
#plt.hist(v_hist_midT_FeII,xbins,histtype = 'step', alpha=0.7,color='red',label ='Fe II',weights=m_hist_midT_FeII)
#plt.hist(v_hist_midT_CII,xbins,histtype = 'step', alpha=0.9,color='red',label ='C II',weights=m_hist_midT_CII)
plt.hist(v_hist_midT_OVI,xbins,histtype = 'step', alpha=0.9,linewidth=4.0,color='black',label ='O VI',weights=m_hist_midT_OVI)
plt.legend(loc='upper right',fontsize=32)
matplotlib.rc('xtick', labelsize=25) 
matplotlib.rc('ytick', labelsize=25)
ax.set_xlabel('velocity $km/s$',fontsize=36)
ax.set_ylabel('Total mass $M_{\odot}$',fontsize=36)
ax.set_xlim(0,500)
plt.show()

plt.savefig(dnameout+'1D_vel_mass_OVI'+str(i)+'.png', dpi=300)
plt.close(fig)

In [32]:
xbins = np.linspace(0,1200,200)
fig = plt.figure(figsize=(15,15))
ax = fig.add_axes([0.19,0.17,0.58,0.75])
v_hist_midT_SiII=v_hist[midT_SiII]
m_hist_midT_SiII=m_hist[midT_SiII]
plt.hist(v_hist_midT_SiII,xbins,histtype = 'step', alpha=0.9,linewidth=4.0,color='red',label ='Si II',weights=m_hist_midT_SiII)
plt.legend(loc='upper right',fontsize=32)
matplotlib.rc('xtick', labelsize=25) 
matplotlib.rc('ytick', labelsize=25)
ax.set_xlabel('velocity $km/s$',fontsize=36)
ax.set_ylabel('Total mass $M_{\odot}$',fontsize=36)
ax.set_xlim(0,500)
plt.show()

plt.savefig(dnameout+'1D_vel_mass_SiII'+str(i)+'.png', dpi=300)
plt.close(fig)

In [33]:
xbins = np.linspace(0,1200,100)
fig = plt.figure(figsize=(8,7))
ax = fig.add_axes([0.19,0.17,0.58,0.75])

plt.hist(vx_260260,xbins,histtype = 'step', alpha=0.9,color='red',label ='Si II',weights=N_SiII_x)
plt.legend(loc='upper right')
ax.set_xlabel('velocity $km/s$')
ax.set_ylabel('Total mass $M_{\odot}$')
plt.show()

ValueError: weights should have the same shape as x

In [ ]:
bins = np.linspace(0,1200,100)
fig = plt.figure(figsize=(8,7))
ax = fig.add_axes([0.19,0.17,0.58,0.75])
plt.hist(vx_260260,xbins,histtype = 'step', alpha=0.5,color='blue',label ='Fe II',weights=N_FeII_x)
plt.legend(loc='upper right')
ax.set_xlabel('velocity $km/s$')
ax.set_ylabel('Total mass $M_{\odot}$')
plt.show()

In [ ]:
xbins = np.linspace(0,1200,100)
fig = plt.figure(figsize=(8,7))
ax = fig.add_axes([0.19,0.17,0.58,0.75])
plt.hist(vx_260260,xbins,histtype = 'step', alpha=0.5,color='green',label ='O VI',weights=N_OVI_x)
plt.legend(loc='upper right')
ax.set_xlabel('velocity $km/s$')
ax.set_ylabel('Total mass $M_{\odot}$')
plt.show()

In [ ]:
values_Si_test=np.zeros((ny,nz,1200))
for j in range(0,ny):
    for k in range(0,nz):
        N_Si_II=n_Si[:,j,k]
        vel_Si=vx[:,j,k]
        values_Si_test[j,k,:],b=np.histogram(vel_Si,bins=1200,weights=N_Si_II,range=(0,1200))

In [ ]:
values=np.zeros((ny,nz,100))
for j in range(0,ny):
    for k in range(0,nz):
        N_Fe_II=n_Fe[:,j,k]
        vel=vx[:,j,k]
        values[j,k,:],b=np.histogram(vel,bins=100,weights=N_Fe_II,range=(0,1200))

In [ ]:
values_Si=np.zeros((ny,nz,100))
for j in range(0,ny):
    for k in range(0,nz):
        N_Si_II=n_Si[:,j,k]
        vel_Si=vx[:,j,k]
        values_Si[j,k,:],b=np.histogram(vel_Si,bins=100,weights=N_Si_II,range=(0,1200))

In [ ]:
np.save('Si_test',values_Si_test)

In [ ]:
values_O=np.zeros((ny,nz,100))
for j in range(0,ny):
    for k in range(0,nz):
        N_O_VI=n_O[:,j,k]
        vel_O=vx[:,j,k]
        values_O[j,k,:],b=np.histogram(vel_O,bins=100,weights=N_O_VI,range=(0,1200))

In [ ]:
values_C=np.zeros((ny,nz,100))
for j in range(0,ny):
    for k in range(0,nz):
        N_C_II=n_C[:,j,k]
        vel_C=vx[:,j,k]
        values_C[j,k,:],b=np.histogram(vel_C,bins=100,weights=N_C_II,range=(0,1200))

In [ ]:
np.save('Tau_SiII',values_Si)
np.save('Tau_OVI',values_O)
np.save('Tau_CII',values_C)

In [ ]:
me=9.10e-28
c=3e18
dv=100
dr=4.94e20 #in cm

In [ ]:
#lst=np.array(TSiII[:,0])/512

In [ ]:
#np.sum(TFeII[260,260,:])*dr

In [ ]:
np.where(n_Si>1e-18)

In [ ]:
np.sum(n_O[:,260,260])*dr

In [ ]:
np.sum(TSiII[300,280,:])*dr

In [ ]:
np.sum(TOVI[260,260,:])*dr

In [ ]:
np.max(vx)

In [ ]:
np.save('N_SiII',n_Si)
np.save('N_OVI',n_O)

In [ ]:
np.save('vx',vx)

In [ ]:
np.min(vx)